In [161]:
import pandas as pd
import numpy as np

In [162]:
df = pd.read_csv("test-data/csvtest.csv")

In [163]:
df

,x,y,z
0,36,78,96
1,37,79,95
2,36,78,94
3,35,79,93
4,36,79,93
5,36,80,92
6,36,81,91
7,36,82,90
8,37,82,89
9,37,83,88


In [164]:
df1=df
df1['key'] = 1
v = df1.merge(df1, on='key').drop('key', 1)

/var/folders/sd/1vc_q83x5rn9jjrd0x47_cc00000gn/T/ipykernel_42895/2436364694.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  v = df1.merge(df1, on='key').drop('key', 1)


In [165]:
df1

,x,y,z,key
0,36,78,96,1
1,37,79,95,1
2,36,78,94,1
3,35,79,93,1
4,36,79,93,1
5,36,80,92,1
6,36,81,91,1
7,36,82,90,1
8,37,82,89,1
9,37,83,88,1


In [166]:
v

,x_x,y_x,z_x,x_y,y_y,z_y
0,36,78,96,36,78,96
1,36,78,96,37,79,95
2,36,78,96,36,78,94
3,36,78,96,35,79,93
4,36,78,96,36,79,93
...,...,...,...,...,...,...
95,37,83,88,36,80,92
96,37,83,88,36,81,91
97,37,83,88,36,82,90
98,37,83,88,37,82,89


In [167]:
a, b = np.split(v.values, 2, axis=1)
n=np.linalg.norm(a-b, axis=1)

In [168]:
n

array([0.        , 1.73205081, 2.        , 3.31662479, 3.16227766,
       4.47213595, 5.83095189, 7.21110255, 8.1240384 , 9.48683298,
       1.73205081, 0.        , 1.73205081, 2.82842712, 2.23606798,
       3.31662479, 4.58257569, 5.91607978, 6.70820393, 8.06225775,
       2.        , 1.73205081, 0.        , 1.73205081, 1.41421356,
       2.82842712, 4.24264069, 5.65685425, 6.4807407 , 7.87400787,
       3.31662479, 2.82842712, 1.73205081, 0.        , 1.        ,
       1.73205081, 3.        , 4.35889894, 5.38516481, 6.70820393,
       3.16227766, 2.23606798, 1.41421356, 1.        , 0.        ,
       1.41421356, 2.82842712, 4.24264069, 5.09901951, 6.4807407 ,
       4.47213595, 3.31662479, 2.82842712, 1.73205081, 1.41421356,
       0.        , 1.41421356, 2.82842712, 3.74165739, 5.09901951,
       5.83095189, 4.58257569, 4.24264069, 3.        , 2.82842712,
       1.41421356, 0.        , 1.41421356, 2.44948974, 3.74165739,
       7.21110255, 5.91607978, 5.65685425, 4.35889894, 4.24264

## Distance Computation Function:


## Create generator function since brute force approach is extremely expensive computationally:


if `df` is of the form:<br><br>
df=<br>
&emsp;&emsp;x&emsp;y&emsp;z<br>
0&emsp; x0  y0  z0<br>
1&emsp;  x1  y1  z1<br>
2&emsp;  x2  y2  z2<br>
.<br>
.<br>
.<br>

the we could get the associated distances between each node and every other node by the operations:<br>
df['key'] = 1<br>
We create a `Cartersian product` from the dataframe nodes with the following:<br><br>
df1 = df.merge(df, on='key').drop('key', 1)<br><br>
a, b = np.split(df1.values, 2, axis=1)<br>
np.linalg.norm(a-b, axis=1)<br><br>

However, requires $O(N^2)$ memory to create the cartesian product. This can become an issue as the matrix grows (as number of nodes grows).<br>
To avoid this to some extend, we can instead create a generator that does the same thing, at the cost of some speed/performance.

## Distance Generator function:

In [169]:
def c2d(coordinatesDictionary:dict,timestep,moleculeID):
    return pd.DataFrame(coordinatesDictionary["structured-data"][timestep][moleculeID]).T

def atom1ToOrigin(atomsdf):
    first_row = atomsdf.iloc[[0]].values[0]
    return atomsdf.apply(lambda row: row - first_row, axis=1)

def node2node_distances_dataframe(atomsdf):
    '''returns N x N dataframe where N=natoms'''
    def node2node_distances(atomsdf:object):
        def distance_generator(atomsdf:object):
            '''lazy distance computation generator'''
            coordinates = atomsdf.values
            for x in coordinates:
                for y in coordinates:
                    yield np.linalg.norm(x - y)
            return
        return np.array(list(distance_generator(atomsdf))).reshape(len(atomsdf),len(atomsdf)) #reshape because we return a (len(atomsdf)xlen(atomsdf))x1 array otherwise. So we group len(n) elements at a time
    
    def node_distances_dataframe(atomsdf:object):
        distances_matrix = node2node_distances(atomsdf)
        return pd.DataFrame(distances_matrix)
        
    return node_distances_dataframe(atomsdf)

In [170]:
distances_matrix = node2node_distances_dataframe(df)

In [171]:
distances_matrix

,0,1,2,3,4,5,6,7,8,9
0,0.000000,1.732051,2.000000,3.316625,3.162278,4.472136,5.830952,7.211103,8.124038,9.486833
1,1.732051,0.000000,1.732051,2.828427,2.236068,3.316625,4.582576,5.916080,6.708204,8.062258
2,2.000000,1.732051,0.000000,1.732051,1.414214,2.828427,4.242641,5.656854,6.480741,7.874008
3,3.316625,2.828427,1.732051,0.000000,1.000000,1.732051,3.000000,4.358899,5.385165,6.708204
4,3.162278,2.236068,1.414214,1.000000,0.000000,1.414214,2.828427,4.242641,5.099020,6.480741
5,4.472136,3.316625,2.828427,1.732051,1.414214,0.000000,1.414214,2.828427,3.741657,5.099020
6,5.830952,4.582576,4.242641,3.000000,2.828427,1.414214,0.000000,1.414214,2.449490,3.741657
7,7.211103,5.916080,5.656854,4.358899,4.242641,2.828427,1.414214,0.000000,1.414214,2.449490
8,8.124038,6.708204,6.480741,5.385165,5.099020,3.741657,2.449490,1.414214,0.000000,1.414214
9,9.486833,8.062258,7.874008,6.708204,6.480741,5.099020,3.741657,2.449490,1.414214,0.000000
